In [1]:
!pip install git+https://github.com/jsantoso2/keras-ocr.git#egg=keras-ocr

  Cloning https://github.com/jsantoso2/keras-ocr.git to /tmp/pip-install-eiilb35f/keras-ocr
  Running command git clone -q https://github.com/jsantoso2/keras-ocr.git /tmp/pip-install-eiilb35f/keras-ocr
     |████████████████████████████████| 9.5MB 9.3MB/s 
     |████████████████████████████████| 808kB 43.7MB/s 
     |████████████████████████████████| 133kB 52.6MB/s 
  Created wheel for keras-ocr: filename=keras_ocr-0.8.4.post0.dev7-cp36-none-any.whl size=150195 sha256=6bf29954ef8dc64ba0549b9920f4c3a482415deb47550032ab3826c50dd8185d
  Stored in directory: /tmp/pip-ephem-wheel-cache-80tardr9/wheels/26/84/b0/c9fecf52482086189ced4cd35556f70a2117dded59bc4712b5
  Created wheel for validators: filename=validators-0.15.0-cp36-none-any.whl size=18371 sha256=bccf05e693773b34cf4e9298271fe53a3a1feb07c835c2fbf6f2f7120d71c98b
  Stored in directory: /root/.cache/pip/wheels/56/48/64/66c29f8eaf756b780549db047057e1563dfd2d2ce8e3d316c1
Successfully built keras-ocr validators


In [2]:
!nvidia-smi

Tue Jun 16 19:09:57 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
from google.colab.patches import cv2_imshow
import time

import keras_ocr

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

'Amenity Detection'    colab		 'Getting started.pdf'
'Car Classification'  'Colab Notebooks'   LicensePlateDetector


**Make Prediction**

In [6]:
# YOLO CONFIGURATIONS
net = cv2.dnn.readNetFromDarknet('/mydrive/LicensePlateDetector/backup/yolov3_custom_train.cfg', '/mydrive/LicensePlateDetector/backup/yolov3_custom_train_1800.weights')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL)   #GPU = cv2.dnn.DNN_TARGET_OPENCL_FP16, CPU = DNN_TARGET_CPU

In [7]:
# YOLO CONFIGURATIONS
inpWidth = 416  #608     #Width of network's input image
inpHeight = 416 #608     #Height of network's input image
confThreshold = 0.5  #Confidence threshold
nmsThreshold = 0.4  #Non-maximum suppression threshold

classes = ['LP']   #This is the class labels

In [8]:
# keras-ocr will automatically download pretrained weights for the detector and recognizer.
detector = keras_ocr.detection.Detector(weights='clovaai_general', 
                                        load_from_torch=False, 
                                        optimizer='adam', 
                                        backbone_name='vgg',
				                                weights_path_local = '/mydrive/LicensePlateDetector/craft_mlt_25k.h5')
recognizer = keras_ocr.recognition.Recognizer(alphabet=None, 
                                              weights='kurapan', 
                                              build_params=None, 
                                              weights_path = '/mydrive/LicensePlateDetector/crnn_kurapan.h5')


pipeline = keras_ocr.pipeline.Pipeline(detector=detector, recognizer=recognizer)

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [9]:
# Get the names of the output layers
def getOutputsNames(net):
    # Get the names of all the layers in the network
    layersNames = net.getLayerNames()
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [10]:
# Draw the predicted bounding box
def drawPred(frame, classId, conf, left, top, right, bottom):

    # get cropped LP for text preprocessing
    crop_img = frame[top:bottom, left:right]

    # invert image  (CANNOT convert to grayscale because the KERAS OCR expects 3 channels)
    crop_img = cv2.bitwise_not(crop_img)

    # expand image size for better recognition
    (h, w) = crop_img.shape[:2]
    crop_img = cv2.resize(crop_img, (w*2, h*2), interpolation = cv2.INTER_AREA)

    start = time.time()
    prediction_groups = pipeline.recognize([crop_img], detection_kwargs={'detection_threshold': 0.5, 'text_threshold': 0.4, 'link_threshold': 0.4}, recognition_kwargs={})
    end = time.time()
    print('Text Processing time: ', (end - start) * 100, ' ms')

    #print(prediction_groups[0])

    # only take top 3 to prevent from reading bottom line which is the expiry date
    if len(prediction_groups[0]) >= 3:
        predictions = prediction_groups[0][0:3]
    else:
        predictions = prediction_groups[0]

    # extract left starting coordinates and rearrange to read from left to right
    left_start_coor = [elem[1][0][0] for elem in predictions]
    sort_order = np.argsort(left_start_coor)
    predictions = [x for _,x in sorted(zip(sort_order,predictions))]

    # get only the text
    prediction_text = [x[0].upper() for x in predictions]
    #print(prediction_text)

    ## IF LEN == 3 THEN WE CAN PERFORM VALIDATIONS to HELP...
    if len(prediction_text) == 3:
      # Ensure that first character is originating from license plate
      all_regions = ['BL', 'BB', 'BK', 'BA', 'BM', 'BP', 'BG', 'BN', 'BE', 'BD', 'BH', 'B', 'D', 'F', 'T', 'E',
                    'Z', 'H', 'G', 'K', 'R', 'AA', 'AD', 'AB', 'L', 'W', 'N', 'P', 'AG', 'AE', 'S', 'M', 'DK',
                    'EA', 'DH', 'EB', 'ED', 'KB', 'DA', 'KH', 'KT', 'KU', 'DB', 'DL', 'DM', 'DN', 'DD', 'DC',
                    'DT', 'DE', 'DG', 'DS', 'PB']
      
      # check validation
      if prediction_text[0].upper() in all_regions:
          pass
      else:
          # if two characters check first and second characters
          curr_reading = prediction_text[0].upper()
          if len(curr_reading) == 2:
              if curr_reading[0] in all_regions:
                  prediction_text[0] = curr_reading[0]
              elif curr_reading[1] in all_regions:
                  prediction_text[0] = curr_reading[1]
              else:
                  pass
          else:
              pass

      # # check vehicle type 
      #vehicle_type = 'Car'
      # if int(prediction_text[1]) >= 1 and int(prediction_text[1]) <= 1999:
      #     vehicle_type = 'Car'
      # elif int(prediction_text[1]) >= 2000 and int(prediction_text[1]) <= 6999:
      #     vehicle_type = 'Motorcycle'
      # elif int(prediction_text[1]) >= 7000 and int(prediction_text[1]) <= 7999:
      #     vehicle_type = 'Bus'
      # elif int(prediction_text[1]) >= 8000 and int(prediction_text[1]) <= 9999:
      #     vehicle_type = 'Truck'


    # display text annotations 
    #fig, ax = plt.subplots(1,1)
    #keras_ocr.tools.drawAnnotations(image=crop_img, predictions=predictions, ax=ax)
    #plt.show()

    # Draw a bounding box for License Plate
    cv2.rectangle(frame, (left, top), (right, bottom), (255, 255, 255), 3)

    label = '%.2f' % conf

    # Get the label for the class name and its confidence
    if classes:
        assert(classId < len(classes))
        label = '%s:%s' % (classes[classId], label)
    
    # Label license Plate
    sep = ' '
    final_text = sep.join(prediction_text)
    labelSize, baseLine = cv2.getTextSize(final_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top1 = max(top, labelSize[1])
    cv2.rectangle(frame, (left, top1 - round(1.5*labelSize[1]) - 5), (left + round(1.5*labelSize[0]), top1 + baseLine - 5), (0, 0, 0), cv2.FILLED)
    cv2.putText(frame, final_text, (left, top1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)

    #Display the label at the bottom of the bounding box (LicensePlate)
    labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top = max(top, labelSize[1])
    cv2.rectangle(frame, (left, top - round(1.5*labelSize[1]) + (bottom - top + 20)), (left + round(1.5*labelSize[0]), top + baseLine + (bottom - top + 20)), (0, 0, 0), cv2.FILLED)
    cv2.putText(frame, label, (left, top + (bottom - top + 20)), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255,255,255), 2)

    #print('from drawpred')
    #cv2_imshow(frame)

In [11]:
# Remove the bounding boxes with low confidence using non-maxima suppression
def postprocess(frame, outs):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]

    #classIds = []
    #confidences = []
    #boxes = []
    # Scan through all the bounding boxes output from the network and keep only the
    # ones with high confidence scores. Assign the box's class label as the class with the highest score.
    classIds = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > 0.1: #confThreshold:
                center_x = int(detection[0] * frameWidth)
                center_y = int(detection[1] * frameHeight)
                width = int(detection[2] * frameWidth)
                height = int(detection[3] * frameHeight)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])

    # Perform non maximum suppression to eliminate redundant overlapping boxes with
    # lower confidences.
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    for i in indices:
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        drawPred(frame, classIds[i], confidences[i], left, top, left + width, top + height)

In [12]:
def main_generate_prediction(input_path, output_path):
    if input_path != 'webcam':
      file_name, ext = os.path.splitext(input_path)

    # image file
    if ext.lower() == '.jpeg' or ext.lower() == '.jpg' or ext.lower() == '.png':
        cap = cv2.VideoCapture(input_path)

    # video file
    else:
        # mp4 video file
        if ext.lower() == '.mp4':
          cap = cv2.VideoCapture(input_path)
        # webcam
        else:
          cap = cv2.VideoCapture(0)

        vid_writer = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc('M','J','P','G'), 30, (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    # Process Images and Video
    while cv2.waitKey(1) < 0:
      # get frame from the video
      hasFrame, frame = cap.read()

      # Stop the program if reached end of video
      if not hasFrame:
          print("Done processing !!!")
          cv2.waitKey(3000)
          break

      # Create a 4D blob from a frame.
      blob = cv2.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)

      # Sets the input to the network
      net.setInput(blob)

      # Runs the forward pass to get output of the output layers
      outs = net.forward(getOutputsNames(net))

      # Remove the bounding boxes with low confidence
      postprocess(frame, outs)

      # Put efficiency information. The function getPerfProfile returns the overall time for inference(t) and the timings for each of the layers(in layersTimes)
      t, _ = net.getPerfProfile()
      label = 'Inference time: %.2f ms' % (t * 1000.0 / cv2.getTickFrequency())
      print(label)

      # Write the frame with the detection boxes
      if ext.lower() == '.jpeg' or ext.lower() == '.jpg' or ext.lower() == '.png':
          cv2.imwrite(output_path, frame.astype(np.uint8));
          #cv2_imshow(frame.astype(np.uint8))
      else:
          vid_writer.write(frame.astype(np.uint8))

In [13]:
main_generate_prediction('/mydrive/LicensePlateDetector/test.jpg', '/mydrive/LicensePlateDetector/test_pred.jpg')

Text Processing time:  1028.000283241272  ms
Inference time: 4775.87 ms
Done processing !!!


In [14]:
main_generate_prediction('/mydrive/LicensePlateDetector/test2_cropped.jpg', '/mydrive/LicensePlateDetector/test2_cropped_pred.jpg')

Text Processing time:  182.44476318359375  ms
Inference time: 254.17 ms
Done processing !!!


In [15]:
main_generate_prediction('/mydrive/LicensePlateDetector/test3.jpg', '/mydrive/LicensePlateDetector/test3_pred.jpg')

Text Processing time:  238.14966678619385  ms
Inference time: 247.88 ms
Done processing !!!


In [16]:
main_generate_prediction('/mydrive/LicensePlateDetector/test4.jpg', '/mydrive/LicensePlateDetector/test4_pred.jpg')

Text Processing time:  198.21252822875977  ms
Inference time: 255.54 ms
Done processing !!!


In [17]:
start = time.time()
main_generate_prediction('/mydrive/LicensePlateDetector/test_video2.mp4', '/mydrive/LicensePlateDetector/test_video2_pred.avi')
end = time.time()
print('Total Time: ', end - start, ' s')

Inference time: 358.33 ms
Text Processing time:  143.00124645233154  ms
Inference time: 251.10 ms
Text Processing time:  112.34910488128662  ms
Inference time: 251.95 ms
Text Processing time:  7.416749000549316  ms
Inference time: 250.76 ms
Text Processing time:  8.009624481201172  ms
Inference time: 244.88 ms
Inference time: 249.97 ms
Inference time: 254.12 ms
Inference time: 252.56 ms
Inference time: 251.73 ms
Text Processing time:  130.6781768798828  ms
Inference time: 252.75 ms
Text Processing time:  114.29891586303711  ms
Inference time: 250.39 ms
Inference time: 251.38 ms
Inference time: 251.68 ms
Inference time: 250.31 ms
Text Processing time:  141.10510349273682  ms
Inference time: 253.20 ms
Text Processing time:  130.90810775756836  ms
Inference time: 250.74 ms
Text Processing time:  129.54175472259521  ms
Inference time: 250.79 ms
Text Processing time:  114.44799900054932  ms
Inference time: 252.19 ms
Text Processing time:  128.10134887695312  ms
Inference time: 257.25 ms
Tex

In [18]:
main_generate_prediction('/mydrive/LicensePlateDetector/test_video_orig.mp4', '/mydrive/LicensePlateDetector/test_video_orig_pred.avi')

Text Processing time:  281.08832836151123  ms
Inference time: 345.86 ms
Text Processing time:  265.1589632034302  ms
Inference time: 256.85 ms
Text Processing time:  271.95637226104736  ms
Inference time: 260.38 ms
Text Processing time:  278.10218334198  ms
Inference time: 258.37 ms
Text Processing time:  255.33480644226074  ms
Inference time: 259.47 ms
Text Processing time:  241.9703245162964  ms
Inference time: 256.48 ms
Text Processing time:  234.84420776367188  ms
Inference time: 260.40 ms
Text Processing time:  258.7310552597046  ms
Inference time: 259.03 ms
Text Processing time:  216.32006168365479  ms
Inference time: 263.22 ms
Text Processing time:  221.92020416259766  ms
Inference time: 262.10 ms
Text Processing time:  222.8372097015381  ms
Inference time: 260.25 ms
Text Processing time:  284.07793045043945  ms
Inference time: 264.85 ms
Text Processing time:  279.3090581893921  ms
Inference time: 263.14 ms
Text Processing time:  264.9979829788208  ms
Inference time: 259.62 ms
T

In [19]:
start = time.time()
main_generate_prediction('/mydrive/LicensePlateDetector/sby_video.mp4', '/mydrive/LicensePlateDetector/sby_video_pred.avi')
end = time.time()

Text Processing time:  3.6130428314208984  ms
Inference time: 256.38 ms
Text Processing time:  3.335261344909668  ms
Inference time: 257.28 ms
Text Processing time:  87.02197074890137  ms
Inference time: 259.73 ms
Text Processing time:  78.63340377807617  ms
Inference time: 260.09 ms
Text Processing time:  3.3990859985351562  ms
Inference time: 259.64 ms
Text Processing time:  79.08945083618164  ms
Inference time: 254.27 ms
Text Processing time:  3.3793210983276367  ms
Inference time: 255.69 ms
Text Processing time:  3.8169145584106445  ms
Inference time: 257.69 ms
Text Processing time:  3.3588171005249023  ms
Inference time: 256.12 ms
Text Processing time:  80.47292232513428  ms
Inference time: 259.42 ms
Text Processing time:  80.35147190093994  ms
Inference time: 257.44 ms
Text Processing time:  97.45264053344727  ms
Inference time: 258.13 ms
Text Processing time:  92.41254329681396  ms
Inference time: 258.80 ms
Text Processing time:  3.4862279891967773  ms
Inference time: 255.97 ms


In [20]:
print(end - start)

289.4334542751312
